## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**3058. Friends With No Mutual Friends (Medium)**

**Table: Friends**

| Column Name | Type |
|-------------|------|
| user_id1    | int  |
| user_id2    | int  |

(user_id1, user_id2) is the primary key (combination of columns with unique values) for this table.
Each row contains user id1, user id2, both of whom are friends with each other.

**Write a solution to find all pairs of users who are friends with each other and have no mutual friends.**

Return the result table ordered by user_id1, user_id2 in ascending order.

The result format is in the following example.

**Example 1:**

**Input:**
**Friends table:**
| user_id1 | user_id2 | 
|----------|----------|
| 1        | 2        | 
| 2        | 3        | 
| 2        | 4        | 
| 1        | 5        | 
| 6        | 7        | 
| 3        | 4        | 
| 2        | 5        | 
| 8        | 9        | 

**Output:**
| user_id1 | user_id2 | 
|----------|----------|
| 6        | 7        | 
| 8        | 9        | 

**Explanation:** 
- Users 1 and 2 are friends with each other, but they share a mutual friend with user ID 5, so this pair is not included.
- Users 2 and 3 are friends, they both share a mutual friend with user ID 4, resulting in exclusion, similarly for users 2 and 4 who share a mutual friend with user ID 3, hence not included.
- Users 1 and 5 are friends with each other, but they share a mutual friend with user ID 2, so this pair is not included.
- Users 6 and 7, as well as users 8 and 9, are friends with each other, and they don't have any mutual friends, hence included.
- Users 3 and 4 are friends with each other, but their mutual connection with user ID 2 means they are not included, similarly for users 2 and 5 are friends but are excluded due to their mutual connection with user ID 1.

Output table is ordered by user_id1 in ascending order.

In [0]:
friends_data_3058 = [
    (1, 2),
    (2, 3),
    (2, 4),
    (1, 5),
    (6, 7),
    (3, 4),
    (2, 5),
    (8, 9),
]

friends_columns_3058 = ["user_id1", "user_id2"]
friends_df_3058 = spark.createDataFrame(friends_data_3058, friends_columns_3058)
friends_df_3058.show()

+--------+--------+
|user_id1|user_id2|
+--------+--------+
|       1|       2|
|       2|       3|
|       2|       4|
|       1|       5|
|       6|       7|
|       3|       4|
|       2|       5|
|       8|       9|
+--------+--------+



In [0]:
friends_bidir_3058 = friends_df_3058\
                        .union(
                            friends_df_3058\
                                .select(
                                    col("user_id2").alias("user_id1"),
                                    col("user_id1").alias("user_id2")
                                    )
                                )

In [0]:
friends_list_3058 = friends_bidir_3058\
                        .groupBy("user_id1") \
                            .agg(
                                collect_set("user_id2").alias("friends_set")
                                )

In [0]:
joined_df_3058 = friends_df_3058.alias("f")\
                        .join( friends_list_3058.alias("fl1"), col("f.user_id1") == col("fl1.user_id1"))\
                            .join( friends_list_3058.alias("fl2"), col("f.user_id2") == col("fl2.user_id1"))

In [0]:
joined_df_3058\
    .withColumn( "mutual_friends", array_intersect(col("fl1.friends_set"), col("fl2.friends_set")))\
        .filter(size(col("mutual_friends")) == 0) \
            .select("f.user_id1", "f.user_id2") \
                .orderBy("user_id1", "user_id2").display()

user_id1,user_id2
6,7
8,9
